In [ ]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from kafka import KafkaProducer
import json
import time

# twitter setup
consumer_key = "JbQSXEAARSYk1vQNCp3h4eeTt"
consumer_secret = "lkzoyk8qwBnzNhhE2pv9pk3q7MhJ8LV6KJUZEWBPM0U9gbqZqY"
access_token = "1204687246437363712-cvV0yKDLWRjGxBDy6gRCI20519isjP"
access_token_secret = "xVoFh2S3r7qDFaRrYau4yaefsnoUYTol6e4VRDWdL50w4"


# Creating the authentication object
auth = OAuthHandler(consumer_key, consumer_secret)   
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
# Creating the API object by passing in auth information

topic_name="UCLFinal"  #creiamo un topic


from datetime import datetime, timedelta    #normalizzazione della data
def normalize_timestamp(time):
    mytime = datetime.strptime(time, "%Y-%m-%d %H:%M:%S")
    mytime += timedelta(hours=2)   # the tweets are timestamped in GMT timezone, while I am in +2 timezone
    return (mytime.strftime("%Y-%m-%d %H:%M:%S"))

producer = KafkaProducer(bootstrap_servers='kafka:9092',value_serializer=lambda v: json.dumps(v).encode("utf-8"))

class StdOutListener(StreamListener):
    
    def __init__(self, time_limit=10):
        self.start_time=time.time()
        self.limit = time_limit
        super(StdOutListener, self).__init__() 
        
    def on_status(self,status):
        if (time.time() - self.start_time) < self.limit:
            if hasattr(status,"retweeted_status"):
                    x=True
            else:
                    x=False
            producer.send(topic_name,value={'id': status.id_str,
                                        'user_location':status.user.location,
                                          'language':status.lang,
                                         'timestamp':normalize_timestamp(str(status.created_at)),
                                         'text':status.text,
                                         'hashtags':status.entities['hashtags'], 
                                          'urls':status.entities['urls'],
                                            'user_mentions':status.entities['user_mentions'],
                                         'geo':status.geo,
                                         'coordinates':status.coordinates,
                                         'retweet_count':status.retweet_count,
                                           'retweeted':status.retweeted,
                                           'is_a_retweet': str(x),
                                           'user_followers':status.user.followers_count})
       
                         
    def on_error(self,status):    
        print(status)
        
    def on_exception(self, exception):    
        print(exception)
        return
        
        

stream= Stream(auth,listener=StdOutListener(time_limit = 60*60*7))
stream.filter(track=['#UCL','#ChampionsLeague','#PSGBayern','#UCLFinal','#PSGFCB'])
